# 📄 Resume Retrieval System using RAG + ChromaDB

This project implements a **retrieval-based resume analysis system** using a 
Retrieval-Augmented Generation (RAG) approach without an LLM.

The system works by:
1. Reading multiple resumes (PDFs)
2. Extracting text and cleaning it
3. Applying one chosen **chunking technique**
4. Storing the chunk embeddings into **ChromaDB**
5. Asking predefined questions (from a `.txt` file)
6. Retrieving the most relevant sections from each resume

🔍 This allows the system to answer:
- What technical skills does each candidate have?
- What projects did they work on?
- Do they have internship experience?
- How strong are their qualifications?

📌 Each resume is processed **independently**, ensuring fair comparison across all candidates.



In [1]:
# Install required libraries if not available
!pip install chromadb sentence-transformers pymupdf nltk scikit-learn --quiet

In [2]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

import fitz
import chromadb
from chromadb.utils import embedding_functions
import re
import nltk
from nltk.tokenize import sent_tokenize
from sklearn.metrics import precision_score, recall_score, f1_score

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Chunking Methods

def sentence_chunking(text, max_sentences=5):
    sentences = sent_tokenize(text)
    return [" ".join(sentences[i:i+max_sentences])
            for i in range(0, len(sentences), max_sentences)]

def paragraph_chunking(text):
    return [p for p in text.split("\n") if len(p.strip()) > 0]

def semantic_chunking(text, size=120, overlap=40):
    words = text.split()
    chunks = []
    step = size - overlap
    for i in range(0, len(words), step):
        chunks.append(" ".join(words[i:i+size]))
    return chunks

def sliding_window_chunking(text, chunk_size=120, overlap=40):
    words = text.split()
    chunks = []
    step = chunk_size - overlap
    for i in range(0, len(words), step):
        chunk_words = words[i:i + chunk_size]
        if not chunk_words:
            break
        chunks.append(" ".join(chunk_words))
    return chunks

In [4]:
# Chunking Selection (ONE TIME)

CHUNK_METHOD = None

def choose_chunking_once():
    global CHUNK_METHOD
    if CHUNK_METHOD is not None:
        return CHUNK_METHOD

    print("\n🔹 Select Chunking Method:")
    print("1️⃣ Sentence-based")
    print("2️⃣ Paragraph-based")
    print("3️⃣ Semantic-based")
    print("4️⃣ Sliding Window-based")

    choice = input("Enter 1, 2, 3, or 4: ")

    CHUNK_METHOD = {
        "1": sentence_chunking,
        "2": paragraph_chunking,
        "3": semantic_chunking,
        "4": sliding_window_chunking
    }.get(choice, semantic_chunking)

    return CHUNK_METHOD

In [5]:
# Embedding Selection (ONE TIME)

def choose_embedding():
    print("\n✨ Choose Embedding Model:")
    print("1️⃣ all-MiniLM-L6-v2")
    print("2️⃣ all-mpnet-base-v2")
    print("3️⃣ paraphrase-mpnet-base-v2")

    choice = input("Enter your choice: ")

    return {
        "1": "all-MiniLM-L6-v2",
        "2": "all-mpnet-base-v2",
        "3": "paraphrase-mpnet-base-v2"
    }.get(choice, "all-MiniLM-L6-v2")

In [6]:
# Read Resume

def preprocess(text):
    return re.sub(r"\s+", " ", text).strip()

def read_resume(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += preprocess(page.get_text("text")) + "\n"
    return text

In [7]:
# Create Collection (per CV)

def create_collection(client, model_name, name):
    try:
        client.delete_collection(name)
    except:
        pass

    emb = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name=model_name
    )

    return client.create_collection(name=name, embedding_function=emb)

In [8]:
# Index Resume

def index_resume(collection, pdf_path, base_id):
    global CHUNK_METHOD
    
    if CHUNK_METHOD is None:
        CHUNK_METHOD = choose_chunking_once()

    text = read_resume(pdf_path)
    chunks = CHUNK_METHOD(text)

    ids = [f"{base_id}_{i}" for i in range(len(chunks))]
    collection.add(documents=chunks, ids=ids)

    print(f"📌 Indexed {len(chunks)} chunks → {os.path.basename(pdf_path)}")
    return chunks

In [9]:
# Retrieval

def ask_resume(collection, query, topk=3, show=True):
    results = collection.query(query_texts=[query], n_results=topk)
    ids = results["ids"][0]
    docs = results["documents"][0]
    dists = results["distances"][0]

    print(f"\n🔍 Query: {query}")
    print("--------------------------------------------------")

    if show:
        for i, (doc_id, text, dist) in enumerate(zip(ids, docs, dists), 1):
            print(f"\n⭐ Result #{i}")
            print(f"ID: {doc_id}")
            print(f"Similarity: {1 - dist:.4f}")
            print(f"Text:\n{text[:300]}...")
            print("-" * 50)

    return docs

In [10]:
# Evaluation Layer

def load_ground_truth(path):
    gt = {}
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            q, kws = line.strip().split("|")
            gt[q.strip()] = [k.strip().lower() for k in kws.split(",")]
    return gt

def evaluate_retrieval(retrieved_docs, keywords):
    y_true = [1] * len(retrieved_docs)
    y_pred = []

    for doc in retrieved_docs:
        relevant = any(k in doc.lower() for k in keywords)
        y_pred.append(1 if relevant else 0)

    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return p, r, f1

In [11]:
# Main Pipeline

PDF_FOLDER = "CVs"
QUESTIONS_FILE = "questions.txt"
GROUND_TRUTH_FILE = "ground_truth.txt"

model_name = choose_embedding()
choose_chunking_once()
ground_truth = load_ground_truth(GROUND_TRUTH_FILE)

with open(QUESTIONS_FILE, "r", encoding="utf-8") as f:
    questions = [q.strip() for q in f if q.strip()]

client = chromadb.Client()
collections = {}

print("\n📌 Indexing CVs...\n")

for file in os.listdir(PDF_FOLDER):
    if not file.endswith(".pdf"):
        continue

    cv_name = file.replace(".pdf", "")
    pdf_path = os.path.join(PDF_FOLDER, file)

    collection = create_collection(client, model_name, f"cv_{cv_name}")
    index_resume(collection, pdf_path, cv_name)
    collections[cv_name] = collection

print("\n📌 Starting Retrieval and Evaluation...\n")
# Static Questions + Evaluation

for cv, collection in collections.items():
    print("\n" + "=" * 70)
    print(f"📄 CV: {cv}")
    print("=" * 70)

    for q in questions:
        print(f"\n🔍 Question: {q}")
        docs = ask_resume(collection, q, topk=3)

        if q in ground_truth:
            p, r, f1 = evaluate_retrieval(docs, ground_truth[q])
            print(f"\n📊 Precision: {p:.3f}")
            print(f"📊 Recall:    {r:.3f}")
            print(f"📊 F1-score:  {f1:.3f}")


✨ Choose Embedding Model:
1️⃣ all-MiniLM-L6-v2
2️⃣ all-mpnet-base-v2
3️⃣ paraphrase-mpnet-base-v2

🔹 Select Chunking Method:
1️⃣ Sentence-based
2️⃣ Paragraph-based
3️⃣ Semantic-based
4️⃣ Sliding Window-based

📌 Indexing CVs...



c:\Users\Lenovo\anaconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📌 Indexed 3 chunks → cv.pdf
📌 Indexed 1 chunks → cv_training.pdf
📌 Indexed 5 chunks → resume.pdf
📌 Indexed 4 chunks → ToqaAsedahCV.pdf

📌 Starting Retrieval and Evaluation...


📄 CV: cv

🔍 Question: What are my technical skills?

🔍 Query: What are my technical skills?
--------------------------------------------------

⭐ Result #1
ID: cv_2
Similarity: 0.5265
Text:
Technical Skills Programming languages: Python, JavaScript, C++, Java Frameworks : Node.js, React,Express Database: MySQL , MongoDB Other: GitHub, Trello, Slack, Notion,Jira ,design pattern and Design principle Education An-Najah National University Aug 2021 - 2025 Computer Science Apprenticeship (C...
--------------------------------------------------

⭐ Result #2
ID: cv_0
Similarity: 0.4021
Text:
ibraheem.khdier@gmail.com +(970) 569-049-126 Ibrahim Khdier https://github.com/IbrahimKhdeir https://www.linkedin.com/in/ibrah eem-khdier-0b34a3252/ Experience Software Engineer Intern OppoTrain Jul 2025 - Sep 2025 ● Developed prob

In [12]:
# User-driven Retrieval (Fixed Vector Space) with Evaluation

print("\n" + "=" * 70)
print("🧠 Interactive User Query Mode (type 'exit' to stop)")
print("=" * 70)

while True:
    user_query = input("\n📝 Enter your question: ")
    if user_query.lower() == "exit":
        break

    for cv, collection in collections.items():
        print("\n" + "-" * 60)
        print(f"📄 CV: {cv}")
        print("-" * 60)
        
        # Retrieve top-k documents
        docs = ask_resume(collection, user_query, topk=3)

        # Show evaluation if ground truth exists
        if user_query in ground_truth:
            p, r, f1 = evaluate_retrieval(docs, ground_truth[user_query])
            print(f"\n🔹 Question: {user_query}")
            print(f"📊 Precision: {p:.3f}")
            print(f"📊 Recall:    {r:.3f}")
            print(f"📊 F1-score:  {f1:.3f}")
        else:
            print(f"\n🔹 Question: {user_query}")
            print("⚠ No ground truth available for evaluation.")



🧠 Interactive User Query Mode (type 'exit' to stop)

------------------------------------------------------------
📄 CV: cv
------------------------------------------------------------

🔍 Query: what is your name?
--------------------------------------------------

⭐ Result #1
ID: cv_0
Similarity: 0.2179
Text:
ibraheem.khdier@gmail.com +(970) 569-049-126 Ibrahim Khdier https://github.com/IbrahimKhdeir https://www.linkedin.com/in/ibrah eem-khdier-0b34a3252/ Experience Software Engineer Intern OppoTrain Jul 2025 - Sep 2025 ● Developed problem-solving skills through leetcode. ● Strengthened soft skills such ...
--------------------------------------------------

⭐ Result #2
ID: cv_1
Similarity: 0.0805
Text:
The projects appear to be coursework, demonstrating your understanding of software testing principles and practices. Cap Advisor | GitHub Link | Node.js | Express | MongoDB | EJS | Bootstrap Apr 2024 – May 2024A decision-support web application designed to help capstone students identi